In [2]:
!pip install pip==23.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [3]:
!pip install -q datasets==2.1.0 transformers==4.30.2 tensorflow==2.12.0
import datasets, tensorflow as tf, transformers

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D1024 20:44:06.604796389      15 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D1024 20:44:06.604819102      15 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D1024 20:44:06.604822517      15 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D1024 20:44:06.604825118      15 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D1024 20:44:06.604827887      15 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D1024 20:44:06.604830593      15 config.cc:119]     

In [4]:
# print(tf.__version__)

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sarc-train-balanced/train-balanced.csv
/kaggle/input/sarc-test-balanced/test-balanced.csv


# Setup

In [6]:
sarc_train = pd.read_csv('../input/sarc-train-balanced/train-balanced.csv', delimiter='\t', header=None)
sarc_test = pd.read_csv('../input/sarc-test-balanced/test-balanced.csv', delimiter='\t', header=None)
print(sarc_train.head())
print(sarc_test.head())

   0                                                  1             2  \
0  0                                   Dang dog, thanks    Mattys1174   
1  0  to summon the powers of the flying spaghetti m...    yugiohhero   
2  0       i did that 3rd last 1 by accident last night    yugiohhero   
3  0  He's insane, used him in DC, better than Blake...  masternater6   
4  0  Forgot about him, he's a pretty pointless card...  masternater6   

                 3  4  5  6        7           8  \
0           NHLHUT  1 -1 -1  2017-03  1490927299   
1    CringeAnarchy  1 -1 -1  2017-02  1486349313   
2  TownofSalemgame  8 -1 -1  2017-03  1488514232   
3           NHLHUT  1 -1 -1  2017-02  1486097950   
4           NHLHUT  1 -1 -1  2017-04  1491597507   

                                                   9  
0                             Great guess! Congrats!  
1    can anynone explain what cash me outside means?  
2                       I have seen all these things  
3  ASG Weber review request 

In [7]:
columns = ['label',	'comment',	'author',	'subreddit',	'score',	'ups',	'downs',	'date',	'created_utc',	'parent_comment']

sarc_train.columns = columns
sarc_train.label.value_counts()

label
0    562839
1    562839
Name: count, dtype: int64

In [8]:
sarc_test.columns = columns
sarc_test.label.value_counts()

label
0    141667
1    141667
Name: count, dtype: int64

In [9]:
sarc_train.dropna(subset=['comment'], inplace=True)
sarc_train.dropna(subset=['parent_comment'], inplace=True)

In [10]:
sarc_test.dropna(subset=['comment'], inplace=True)
sarc_test.dropna(subset=['parent_comment'], inplace=True)

In [11]:
sarc_train['word_count'] = sarc_train['comment'].str.split().apply(len)
sarc_train['parent_word_count'] = sarc_train['parent_comment'].str.split().apply(len)
sarc_train

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,word_count,parent_word_count
0,0,"Dang dog, thanks",Mattys1174,NHLHUT,1,-1,-1,2017-03,1490927299,Great guess! Congrats!,3,3
1,0,to summon the powers of the flying spaghetti m...,yugiohhero,CringeAnarchy,1,-1,-1,2017-02,1486349313,can anynone explain what cash me outside means?,16,8
2,0,i did that 3rd last 1 by accident last night,yugiohhero,TownofSalemgame,8,-1,-1,2017-03,1488514232,I have seen all these things,10,6
3,0,"He's insane, used him in DC, better than Blake...",masternater6,NHLHUT,1,-1,-1,2017-02,1486097950,ASG Weber review request Looking to pick him u...,10,22
4,0,"Forgot about him, he's a pretty pointless card...",masternater6,NHLHUT,1,-1,-1,2017-04,1491597507,Modano at least plays to his overall.,19,7
...,...,...,...,...,...,...,...,...,...,...,...,...
1125673,1,So *that's* why I can point my finger and have...,Dark-Star,atheism,1,1,0,2009-03,1238462755,I told my mom i didn't like religious people. ...,15,29
1125674,1,"If the IDF said it, it must be true.",yellowcakewalk,Israel,1,1,0,2009-06,1244921616,Official IDF counts of Palestinian casualties ...,9,22
1125675,1,That's why you have to read widely from Syria ...,glengyron,worldnews,6,6,0,2009-01,1231052237,Read the Egyptian papers for more anti-Hamas r...,18,19
1125676,1,"Yeah, the Palestinian rocket actually left som...",MrG,pics,11,11,0,2009-01,1231368159,"man, those isrealis are really suffering...",17,6


In [12]:
sarc_test['word_count'] = sarc_test['comment'].str.split().apply(len)
sarc_test['parent_word_count'] = sarc_test['parent_comment'].str.split().apply(len)
sarc_test

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,word_count,parent_word_count
0,0,Your mom,Ronan3463,teenagers,1,-1,-1,2017-04,1491694828,What are you doing 3 hours from now?,2,8
1,0,"I'm curious, how much for the White Photons?",NCJoe97,RocketLeagueExchange,1,-1,-1,2017-02,1487092935,"Painted Exotics, wheels, Toppers and more Pain...",8,97
2,0,Khloe Kardashian effect,grpocz,nba,1,-1,-1,2017-03,1489376837,That was some of the worst defensive reboundin...,3,20
3,0,Knowledge is power!,Wolfy_McDerpbutt,darksouls3,1,-1,-1,2017-02,1485992580,almost 400 hours in and I thought you were jus...,3,20
4,0,Get the Prussians (Germans) to pay for a statu...,Warthog_A-10,ireland,1,-1,-1,2017-02,1486081980,I wouldn't mind if we got a bit keener about o...,14,48
...,...,...,...,...,...,...,...,...,...,...,...,...
283329,1,"Damn those oil fairies, why do they keep steal...",M0b1u5,Economics,0,0,0,2009-06,1244700266,World oil reserves fall for first time in decade,16,9
283330,1,Religion must have the answer,Reedzit,science,3,3,0,2009-02,1233605181,I've been searching for the answer for this fo...,5,27
283331,1,oh I thought they were cursing coz they liked ...,darwin_wins,funny,-5,-5,0,2009-02,1235619258,They hated it,13,3
283332,1,My B-52 bombers are in the shop under repair o...,DonnieMarco,reddit.com,2,2,0,2009-04,1239005246,"Humans get tortured at Guantanamo, you make a ...",17,34


In [13]:
sarc_train = sarc_train[(sarc_train['word_count']>3) & (sarc_train['word_count']<50) 
                     & (sarc_train['parent_word_count']>3) & (sarc_train['parent_word_count']<75)]
sarc_train['word_count'].value_counts()
sarc_train['parent_word_count'].value_counts()

parent_word_count
8     46301
7     45462
9     45063
6     43534
10    42523
      ...  
70     1464
71     1414
72     1312
73     1298
74     1160
Name: count, Length: 71, dtype: int64

In [14]:
sarc_test = sarc_test[(sarc_test['word_count']>3) & (sarc_test['word_count']<50) 
                     & (sarc_test['parent_word_count']>3) & (sarc_test['parent_word_count']<75)]
sarc_test['word_count'].value_counts()
sarc_test['parent_word_count'].value_counts()

parent_word_count
7     11570
8     11407
9     11320
6     11053
10    10710
      ...  
68      381
71      375
73      349
72      342
74      319
Name: count, Length: 71, dtype: int64

In [15]:
sarc_train['full_comment'] = sarc_train['parent_comment'].str.cat(sarc_train['comment'], sep=' *&*&*&*&* ')
sarc_train.drop(columns = ['comment',	'author',	'subreddit',	'score',	'ups',	'downs',	'date',	'created_utc',	'parent_comment', 'word_count', 'parent_word_count'], inplace=True)
sarc_train

/tmp/ipykernel_15/317047913.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sarc_train['full_comment'] = sarc_train['parent_comment'].str.cat(sarc_train['comment'], sep=' *&*&*&*&* ')
/tmp/ipykernel_15/317047913.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sarc_train.drop(columns = ['comment',	'author',	'subreddit',	'score',	'ups',	'downs',	'date',	'created_utc',	'parent_comment', 'word_count', 'parent_word_count'], inplace=True)


,label,full_comment
1,0,can anynone explain what cash me outside means...
2,0,I have seen all these things *&*&*&*&* i did t...
3,0,ASG Weber review request Looking to pick him u...
4,0,Modano at least plays to his overall. *&*&*&*&...
9,0,I finished 3 separate boxes of juice *&*&*&*&*...
...,...,...
1125673,1,I told my mom i didn't like religious people. ...
1125674,1,Official IDF counts of Palestinian casualties ...
1125675,1,Read the Egyptian papers for more anti-Hamas r...
1125676,1,"man, those isrealis are really suffering... *&..."


In [16]:
sarc_test['full_comment'] = sarc_test['parent_comment'].str.cat(sarc_test['comment'], sep=' *&*&*&*&* ')
sarc_test.drop(columns = ['comment',	'author',	'subreddit',	'score',	'ups',	'downs',	'date',	'created_utc',	'parent_comment', 'word_count', 'parent_word_count'], inplace=True)
sarc_test

/tmp/ipykernel_15/327488097.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sarc_test['full_comment'] = sarc_test['parent_comment'].str.cat(sarc_test['comment'], sep=' *&*&*&*&* ')
/tmp/ipykernel_15/327488097.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sarc_test.drop(columns = ['comment',	'author',	'subreddit',	'score',	'ups',	'downs',	'date',	'created_utc',	'parent_comment', 'word_count', 'parent_word_count'], inplace=True)


,label,full_comment
4,0,I wouldn't mind if we got a bit keener about o...
5,1,Say goodbye to worker's rights Say hello to th...
7,0,The Mick - S01E07 - The Country Club *&*&*&*&*...
8,0,"Congrats to Trump, the first president to be m..."
9,0,"Guys Brendan said something funny, hoping you ..."
...,...,...
283326,1,Because it is popular to be a democrat right n...
283327,1,The earth has been here for 6000 years and we ...
283329,1,World oil reserves fall for first time in deca...
283330,1,I've been searching for the answer for this fo...


In [17]:
sarc_train.isna().value_counts()

label  full_comment
False  False           848065
Name: count, dtype: int64

In [18]:
sarc_test.isna().value_counts()

label  full_comment
False  False           213428
Name: count, dtype: int64

In [19]:
sarc_train.dropna(inplace=True)
sarc_train.isna().value_counts()

/tmp/ipykernel_15/2346235304.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sarc_train.dropna(inplace=True)


label  full_comment
False  False           848065
Name: count, dtype: int64

In [20]:
sarc_test.dropna(inplace=True)
sarc_test.isna().value_counts()

/tmp/ipykernel_15/2146050555.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sarc_test.dropna(inplace=True)


label  full_comment
False  False           213428
Name: count, dtype: int64

In [21]:
sarc_train['label'].value_counts()

label
1    448362
0    399703
Name: count, dtype: int64

In [22]:
sarc_test['label'].value_counts()

label
1    112699
0    100729
Name: count, dtype: int64

# Some basic text preprocessing

In [23]:
# !pip install spacy
# import spacy

In [24]:
# !python -m spacy download en_core_web_sm
# elp = spacy.load('en_core_web_sm')

In [25]:
sarc_train['full_comment'] = sarc_train['full_comment'].str.lower()

/tmp/ipykernel_15/518157040.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sarc_train['full_comment'] = sarc_train['full_comment'].str.lower()


In [26]:
sarc_test['full_comment'] = sarc_test['full_comment'].str.lower()

/tmp/ipykernel_15/1384167681.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sarc_test['full_comment'] = sarc_test['full_comment'].str.lower()


In [27]:
import re
def remove_special_characters(text):
    # Use regex to remove punctuation, special characters, and numbers
    clean_text = re.sub(r'[^\w\s]', '', text)
    return clean_text

sarc_train['full_comment'] = sarc_train['full_comment'].apply(remove_special_characters)
sarc_test['full_comment'] = sarc_test['full_comment'].apply(remove_special_characters)

/tmp/ipykernel_15/3779270589.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sarc_train['full_comment'] = sarc_train['full_comment'].apply(remove_special_characters)
/tmp/ipykernel_15/3779270589.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sarc_test['full_comment'] = sarc_test['full_comment'].apply(remove_special_characters)


In [28]:
def remove_numbers(text):
    # Use regex to remove numeric characters
    clean_text = re.sub(r'\d', '', text)
    return clean_text

# Apply the function to the 'text_column'
sarc_train['full_comment'] = sarc_train['full_comment'].apply(remove_numbers)
sarc_test['full_comment'] = sarc_test['full_comment'].apply(remove_numbers)

/tmp/ipykernel_15/429139267.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sarc_train['full_comment'] = sarc_train['full_comment'].apply(remove_numbers)
/tmp/ipykernel_15/429139267.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sarc_test['full_comment'] = sarc_test['full_comment'].apply(remove_numbers)


In [29]:
# this took too long

# def remove_stopwords(text):
#     doc = elp(text)
#     tokens_without_stopwords = [token.text for token in doc if not token.is_stop]
#     return ' '.join(tokens_without_stopwords)

# # sarc_test['full_comment'] = sarc_test['full_comment'].apply(lambda x: remove_stopwords(x))

In [30]:
sarc_train

,label,full_comment
1,0,can anynone explain what cash me outside means...
2,0,i have seen all these things i did that rd la...
3,0,asg weber review request looking to pick him u...
4,0,modano at least plays to his overall forgot a...
9,0,i finished separate boxes of juice what type...
...,...,...
1125673,1,i told my mom i didnt like religious people sh...
1125674,1,official idf counts of palestinian casualties ...
1125675,1,read the egyptian papers for more antihamas rh...
1125676,1,man those isrealis are really suffering yeah ...


In [31]:
sarc_test

,label,full_comment
4,0,i wouldnt mind if we got a bit keener about ou...
5,1,say goodbye to workers rights say hello to the...
7,0,the mick se the country club not as good of...
8,0,congrats to trump the first president to be me...
9,0,guys brendan said something funny hoping you g...
...,...,...
283326,1,because it is popular to be a democrat right n...
283327,1,the earth has been here for years and we have...
283329,1,world oil reserves fall for first time in deca...
283330,1,ive been searching for the answer for this for...


# Get samples, get encodings

In [32]:
sarctrain_sm = sarc_train.sample(n=100000, random_state=401)
sarctrain_sm

,label,full_comment
240923,0,i think it is time for all blue state citizens...
785810,1,only time youll ever see this does it have st...
906036,0,til charles manson hoping to incite racial war...
332714,0,people get tired of pretending to respect the ...
295496,1,if you can find close parking sure yeah if th...
...,...,...
402298,1,there are muslim jewish and hindu american pol...
194461,1,packers beg to differ but aaron rodgers was ...
673702,0,i have terabytes of transsexual pornography a...
954846,0,so apparently the only two options here are fa...


In [33]:
sarctrain_sm.label.value_counts()

label
1    52785
0    47215
Name: count, dtype: int64

In [34]:
Xtrain_sm = sarctrain_sm.iloc[:, 1].tolist()
Xtrain_sm

['i think it is time for all blue state citizens to fight tyranny with a boycottthey need to learn that there are consequences for infringing upon democracy and peoples civil liberties   i think they should boycott moving to red states',
 'only time youll ever see this  does it have sticker money',
 'til charles manson hoping to incite racial wars stole his victims wallet and had it planted in the restroom of a gas station in what he thought was a black neighborhood instead of inciting helter skelter as he had hoped it lay there for months until discovered by a cleaning attendant  all he actually accomplished was to show how infrequently gas station bathrooms are cleaned',
 'people get tired of pretending to respect the ideas of idiots  if the last decade was your best effort at pretending we need to get our nation to a fucking acting class',
 'if you can find close parking sure  yeah if they only had some special places for disabled folks to park',
 'halo  artwork  all enemies allies 

In [35]:
ytrain_sm = sarctrain_sm.iloc[:, 0].tolist()
ytrain_sm

[0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,


In [36]:
# import tensorflow as tf
from datasets import load_dataset
from transformers import BertTokenizerFast, TFAutoModelForSequenceClassification
import pickle

In [37]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizerFast.from_pretrained(model_name, use_fast=True)

(…)cased/resolve/main/tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 16.5kB/s]
(…)bert-base-uncased/resolve/main/vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 7.63MB/s]
(…)base-uncased/resolve/main/tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 9.25MB/s]
(…)rt-base-uncased/resolve/main/config.json: 100%|██████████| 570/570 [00:00<00:00, 357kB/s]


In [38]:
def construct_encodings(X, tkzr, max_len, trucation=True, padding=True):
    return tkzr(X, max_length=max_len, truncation=trucation, padding=padding)

MAX_LEN = 125
encodings = construct_encodings(Xtrain_sm, tokenizer, max_len=MAX_LEN)

In [39]:
def construct_tfdataset(encodings, y=None):
    if y:
        return tf.data.Dataset.from_tensor_slices((dict(encodings),y))
    else:
        # this case is used when making predictions on unseen samples after training
        return tf.data.Dataset.from_tensor_slices(dict(encodings))

tfdataset = construct_tfdataset(encodings, ytrain_sm)

In [40]:
VAL_SPLIT = 0.25
BATCH_SIZE = 128

train_size = int(len(Xtrain_sm) * (1-VAL_SPLIT))

tfdataset = tfdataset.shuffle(len(Xtrain_sm))
tfdataset_train = tfdataset.take(train_size)
tfdataset_val = tfdataset.skip(train_size)

tfdataset_train = tfdataset_train.batch(BATCH_SIZE)
tfdataset_val = tfdataset_val.batch(BATCH_SIZE)

# Modeling

## Using Tensorflow

In [41]:
from tensorflow.keras import activations, optimizers, losses
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.losses import BinaryCrossentropy

In [52]:
NUM_LABELS = 2
LEARNING_RATE = 2e2
OPTIMIZER = optimizers.Adam(learning_rate=LEARNING_RATE)
LOSS = losses.BinaryCrossentropy()

In [43]:
# model_name = "bert-base-uncased"
# TFmodel = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS)

In [44]:
class BertBinary(tf.keras.Model):
    def __init__(self, base_model, hidden_dim, freeze_base=True):
        super(BertBinary, self).__init__()
        self.base_model = base_model
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.classifier = tf.keras.Sequential([
            tf.keras.layers.Dense(hidden_dim, activation='relu'),
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.Dense(2)
        ])
        
        if freeze_base:
            self.base_model.trainable = False

    def call(self, inputs, training=False):
        input_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']
        base_model_output = self.base_model(input_ids, attention_mask=attention_mask, training=training)
        pooled_output = base_model_output[0]
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)
        return logits

In [45]:
# For using TPUs

tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:6, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


In [53]:
# let's first train the head

with tpu_strategy.scope():
    TFmodel = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS)
    TFBinary = BertBinary(TFmodel, hidden_dim=256)
    TFBinary.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=['accuracy'])

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
EPOCHS = 10

In [55]:
TFBinary.fit(tfdataset_train, 
             batch_size=BATCH_SIZE, 
             epochs=EPOCHS,
             validation_data=tfdataset_val)

Epoch 1/10


2023-10-24 20:51:57.954492: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-10-24 20:51:58.498650: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


586/586 [==============================] - ETA: 0s - loss: 7.2376 - accuracy: 0.5282

2023-10-24 20:52:38.207927: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-10-24 20:52:38.509232: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


586/586 [==============================] - 74s 81ms/step - loss: 7.2376 - accuracy: 0.5282 - val_loss: 7.1968 - val_accuracy: 0.5306
Epoch 2/10
586/586 [==============================] - 25s 41ms/step - loss: 7.2291 - accuracy: 0.5285 - val_loss: 7.2680 - val_accuracy: 0.5260
Epoch 3/10
586/586 [==============================] - 25s 41ms/step - loss: 7.2659 - accuracy: 0.5261 - val_loss: 7.2453 - val_accuracy: 0.5275
Epoch 4/10
586/586 [==============================] - 24s 41ms/step - loss: 7.2502 - accuracy: 0.5272 - val_loss: 7.1631 - val_accuracy: 0.5328
Epoch 5/10
586/586 [==============================] - 24s 41ms/step - loss: 7.2728 - accuracy: 0.5257 - val_loss: 7.3152 - val_accuracy: 0.5229
Epoch 6/10
586/586 [==============================] - 24s 41ms/step - loss: 7.2356 - accuracy: 0.5281 - val_loss: 7.1913 - val_accuracy: 0.5310
Epoch 7/10
586/586 [==============================] - 26s 44ms/step - loss: 7.2311 - accuracy: 0.5284 - val_loss: 7.2201 - val_accuracy: 0.5291
Epo

In [57]:
# let's try unfreezing the base, and a slower learning rate

LEARNING_RATE = 2e5
OPTIMIZER = optimizers.Adam(learning_rate=LEARNING_RATE)

with tpu_strategy.scope():
    TFBinary = BertBinary(TFmodel, hidden_dim=256, freeze_base=False)
    TFBinary.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=['accuracy'])

In [60]:
TFBinary.fit(tfdataset_train, 
             batch_size=BATCH_SIZE, 
             epochs=EPOCHS,
             validation_data=tfdataset_val)

Epoch 1/10


2023-10-24 20:57:53.271968: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-10-24 20:57:53.783891: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


586/586 [==============================] - ETA: 0s - loss: 7.2088 - accuracy: 0.4856

2023-10-24 20:58:34.069231: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-10-24 20:58:34.365008: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


586/586 [==============================] - 73s 81ms/step - loss: 7.2088 - accuracy: 0.4856 - val_loss: 7.1404 - val_accuracy: 0.4654
Epoch 2/10
586/586 [==============================] - 25s 41ms/step - loss: 7.2271 - accuracy: 0.4861 - val_loss: 7.3342 - val_accuracy: 0.4781
Epoch 3/10
586/586 [==============================] - 25s 41ms/step - loss: 7.2344 - accuracy: 0.4840 - val_loss: 7.2158 - val_accuracy: 0.4703
Epoch 4/10
586/586 [==============================] - 25s 42ms/step - loss: 7.2005 - accuracy: 0.4842 - val_loss: 7.2624 - val_accuracy: 0.4733
Epoch 5/10
586/586 [==============================] - 25s 41ms/step - loss: 7.2516 - accuracy: 0.4848 - val_loss: 7.1361 - val_accuracy: 0.4653
Epoch 6/10
586/586 [==============================] - 25s 41ms/step - loss: 7.2403 - accuracy: 0.4862 - val_loss: 7.2183 - val_accuracy: 0.4707
Epoch 7/10
586/586 [==============================] - 25s 41ms/step - loss: 7.2217 - accuracy: 0.4843 - val_loss: 7.2244 - val_accuracy: 0.4711
Epo

In [61]:
# let's try adding layers to the classification head

class BertBinary2(tf.keras.Model):
    def __init__(self, base_model, hidden_dim, freeze_base=True):
        super(BertBinary2, self).__init__()
        self.base_model = base_model
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.classifier = tf.keras.Sequential([
            tf.keras.layers.Dense(hidden_dim, activation='relu'),
            tf.keras.layers.Dense(hidden_dim, activation='relu'),
            tf.keras.layers.Dense(hidden_dim, activation='relu'),
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.Dense(2)
        ])
        
        if freeze_base:
            self.base_model.trainable = False

    def call(self, inputs, training=False):
        input_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']
        base_model_output = self.base_model(input_ids, attention_mask=attention_mask, training=training)
        pooled_output = base_model_output[0]
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)
        return logits

In [64]:
# let's freeze the base and train on the head only

LEARNING_RATE = 2e2
OPTIMIZER = optimizers.Adam(learning_rate=LEARNING_RATE)

with tpu_strategy.scope():
    TFmodel = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS)
    TFBinary2 = BertBinary2(TFmodel, hidden_dim=256)
    TFBinary2.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=['accuracy'])

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [65]:
TFBinary2.fit(tfdataset_train, 
             batch_size=BATCH_SIZE, 
             epochs=EPOCHS,
             validation_data=tfdataset_val)

Epoch 1/10


2023-10-24 21:04:46.002480: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-10-24 21:04:46.607139: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


586/586 [==============================] - ETA: 0s - loss: 7.2419 - accuracy: 0.4730

2023-10-24 21:05:28.164463: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-10-24 21:05:28.477895: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


586/586 [==============================] - 75s 82ms/step - loss: 7.2419 - accuracy: 0.4730 - val_loss: 7.1919 - val_accuracy: 0.4690
Epoch 2/10
586/586 [==============================] - 25s 42ms/step - loss: 7.2391 - accuracy: 0.4722 - val_loss: 7.2410 - val_accuracy: 0.4722
Epoch 3/10
586/586 [==============================] - 25s 41ms/step - loss: 7.2606 - accuracy: 0.4737 - val_loss: 7.2845 - val_accuracy: 0.4751
Epoch 4/10
586/586 [==============================] - 25s 42ms/step - loss: 7.2334 - accuracy: 0.4720 - val_loss: 7.2446 - val_accuracy: 0.4725
Epoch 5/10
586/586 [==============================] - 25s 42ms/step - loss: 7.2191 - accuracy: 0.4713 - val_loss: 7.2011 - val_accuracy: 0.4696
Epoch 6/10
586/586 [==============================] - 25s 41ms/step - loss: 7.2442 - accuracy: 0.4726 - val_loss: 7.2226 - val_accuracy: 0.4710
Epoch 7/10
586/586 [==============================] - 25s 42ms/step - loss: 7.2504 - accuracy: 0.4726 - val_loss: 7.2710 - val_accuracy: 0.4742
Epo

## with Native Pytorch

In [ ]:
# import torch
# import torch.nn as nn
# from torch.optim import Adam

In [ ]:
# NUM_LABELS = 2
# LEARNING_RATE = 1e3

In [ ]:
# from transformers import AutoModelForSequenceClassification

# HFmodel = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS)

# HFnames = [name for name, param in HFmodel.named_parameters()]
# freeze_names = HFnames[:-2]
# for name, param in HFmodel.named_parameters():
#     if name in freeze_names:
#         param.requires_grad = False    

# HFmodel.classifier = nn.Sequential(
#     nn.Linear(HFmodel.config.hidden_size, NUM_LABELS),
#     nn.Sigmoid()
# )

# optimizer = Adam(HFmodel.parameters(), lr=LEARNING_RATE)
# criterion = nn.BCELoss()

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# HFmodel.to(device)
# criterion.to(device)

I didn't finish figuring out how to use the Pytorch interface. I would need to figure out how to get my data in a dataloader instead of tfdataset.

In [ ]:
# num_epochs = 10
# for epoch in range(num_epochs):
#     for batch in dataloader:
#         inputs, targets = batch
#         inputs, targets = inputs.to(device), targets.to(device)  # Move batch to the appropriate device
        
#         # Forward pass
#         outputs = model(inputs)
        
#         # Compute loss
#         loss = criterion(outputs, targets)
        
#         # Backpropagation and optimization
#         optimizer.zero_grad()  # Clear gradients from previous iteration
#         loss.backward()  # Backpropagation
#         optimizer.step()  # Update weights
        
#         # Print or log training statistics if desired
#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')